### Select patients for CFR model: Split patients in train, val and test sets ###

In [1]:
import os
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [2]:
cfr_data_root = os.path.normpath('/mnt/obi0/andreas/data/cfr')
cfr_meta_date = '200202'
meta_dir = os.path.join(cfr_data_root, 'metadata_'+cfr_meta_date)
print(meta_dir)

/mnt/obi0/andreas/data/cfr/metadata_200202


In [3]:
match_view_filename = '210_getStressTest_match365_files_BWH_'+cfr_meta_date+'.parquet'
files_cfr = pd.read_parquet(os.path.join(meta_dir, match_view_filename))

print('Total number of patients {}'.format(len(files_cfr.mrn.unique())))
print('Total number of studies  {}'.format(len(files_cfr.study.unique())))
print('Total number of files    {}'.format(len(files_cfr.filename.unique())))

files_cfr.head()

Total number of patients 1792
Total number of studies  2921
Total number of files    131290


,mrn,study,echo_study_date,reportID,days_post_cfr,subjectid,report_number,cfr_study_date,cfr_report_date,cfr,filename,dir,datetime,fileid,institution,model,manufacturer,index,frame_time,number_of_frames,heart_rate,deltaX,deltaY,view_a2c,view_a3c,view_a4c,view_plax,view_psax,view_other,sum_views,max_view
0,9241,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e,2010-04-02,100820,64,121,EVS0228610,2010-01-28,2010-01-26,1.133683,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,/mnt/obi0/phi/echo/npyFiles/BWH/4b7b/4b7b463ae...,2010-04-02 07:53:16,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,BWH,Vivid7,GE Vingmed Ultrasound,0.0,40.089256,56.0,85.0,0.027944,0.027944,9.762369e-01,2.330583e-04,3.340996e-03,1.106907e-04,7.710441e-04,1.930706e-02,1.0,view_a2c
1,9241,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e,2010-04-02,100820,64,121,EVS0228610,2010-01-28,2010-01-26,1.133683,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,/mnt/obi0/phi/echo/npyFiles/BWH/4b7b/4b7b463ae...,2010-04-02 07:53:16,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,BWH,Vivid7,GE Vingmed Ultrasound,0.0,40.064101,53.0,88.0,0.017671,0.017671,1.080687e-06,2.221534e-07,1.344890e-06,9.993535e-01,6.972564e-06,6.367801e-04,1.0,view_plax
2,9241,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e,2010-04-02,100820,64,121,EVS0228610,2010-01-28,2010-01-26,1.133683,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,/mnt/obi0/phi/echo/npyFiles/BWH/4b7b/4b7b463ae...,2010-04-02 07:53:16,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,BWH,Vivid7,GE Vingmed Ultrasound,0.0,48.333332,46.0,107.0,0.076726,0.076726,2.892548e-11,2.382341e-09,3.241086e-09,1.000000e+00,2.436432e-10,4.737633e-10,1.0,view_plax
3,9241,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e,2010-04-02,100820,64,121,EVS0228610,2010-01-28,2010-01-26,1.133683,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,/mnt/obi0/phi/echo/npyFiles/BWH/4b7b/4b7b463ae...,2010-04-02 07:53:16,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,BWH,Vivid7,GE Vingmed Ultrasound,0.0,40.450451,38.0,84.0,0.043312,0.043312,3.644072e-10,1.350347e-10,1.540048e-07,1.887208e-10,1.837416e-07,9.999996e-01,1.0,view_other
4,9241,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e,2010-04-02,100820,64,121,EVS0228610,2010-01-28,2010-01-26,1.133683,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,/mnt/obi0/phi/echo/npyFiles/BWH/4b7b/4b7b463ae...,2010-04-02 07:53:16,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,BWH,Vivid7,GE Vingmed Ultrasound,0.0,40.119048,57.0,85.0,0.020879,0.020879,1.801199e-13,9.366453e-13,6.398218e-12,3.151624e-12,6.481914e-12,1.000000e+00,1.0,view_other


### Split the patients in train, validate and test sets ###
Although each view might have a little different patient pupulation distribution, because not all views are in each study. However, we want the same MRNS in each data set and for all views so that we can directly compare the performance of the algorithm for the same patients. We can expand the data frame above to add the splits.

In [4]:
def patientsplit(patient_list):

    train_test_split = 0.86
    train_eval_split = 0.90

    # Split train/test sets
    patient_list_train = np.random.choice(patient_list,
                                          size = int(np.floor(train_test_split*len(patient_list))),
                                          replace = False)
    patient_list_test = list(set(patient_list).difference(patient_list_train))
    train_test_intersection = set(patient_list_train).intersection(set(patient_list_test)) # This should be empty
    print('Intersection of patient_list_train and patient_list_test:', train_test_intersection)

    # Further separate some patients for evaluation
    patient_list_eval = np.random.choice(patient_list_train,
                                         size = int(np.ceil((1-train_eval_split)*len(patient_list_train))),
                                         replace = False)

    patient_list_train = set(patient_list_train).difference(patient_list_eval)
    train_eval_intersection = set(patient_list_train).intersection(set(patient_list_eval))
    print('Intersection of patient_list_train and patient_list_eval:', train_eval_intersection)

    # Show the numbers
    print('total patients:', len(patient_list))
    print()
    print('patients in set:', np.sum([len(patient_list_train),
                                     len(patient_list_eval),
                                     len(patient_list_test)]))
    print()
    print('patients in train:', len(patient_list_train))
    print('patients in eval:', len(patient_list_eval))
    print('patients in test:', len(patient_list_test))

    return patient_list_train, patient_list_eval, patient_list_test

In [5]:
# Get a patient list
patient_list = list(files_cfr.sample(frac=1).mrn.unique())
patient_list_train, patient_list_eval, patient_list_test = patientsplit(patient_list)

patient_split = {'train': patient_list_train,
                 'eval': patient_list_eval,
                 'test': patient_list_test}

print('Patient IDs in train:', len(patient_split['train']))
print('Patient IDs in eval:', len(patient_split['eval']))
print('Patient IDs in test:', len(patient_split['test']))

print()

print('contamination train-test:', set(patient_split['train']).intersection(set(patient_split['test'])))
print('contamination train-eval:', set(patient_split['train']).intersection(set(patient_split['eval'])))
print('contamination eval-test:', set(patient_split['eval']).intersection(set(patient_split['test'])))

Intersection of patient_list_train and patient_list_test: set()
Intersection of patient_list_train and patient_list_eval: set()
total patients: 1792

patients in set: 1792

patients in train: 1386
patients in eval: 155
patients in test: 251
Patient IDs in train: 1386
Patient IDs in eval: 155
Patient IDs in test: 251

contamination train-test: set()
contamination train-eval: set()
contamination eval-test: set()


In [6]:
# Add dset column to files_cfr
split_list = []
for dset in patient_split.keys():
    dset_mrn_list = list(patient_split[dset])
    split_list.append(pd.DataFrame({'mrn': dset_mrn_list,
                                    'dset': [dset]*len(dset_mrn_list)}))
split_df = pd.concat(split_list, ignore_index = True)

files_cfr_dset = files_cfr.merge(right = split_df, on = 'mrn', how = 'left').\
                    sample(frac = 1).\
                    reset_index(drop = True)

In [7]:
files_cfr_dset.head(50)

,mrn,study,echo_study_date,reportID,days_post_cfr,subjectid,report_number,cfr_study_date,cfr_report_date,cfr,filename,dir,datetime,fileid,institution,model,manufacturer,index,frame_time,number_of_frames,heart_rate,deltaX,deltaY,view_a2c,view_a3c,view_a4c,view_plax,view_psax,view_other,sum_views,max_view,dset
0,21722608,49028d1327e3e8cb_4903a584a1bcb5a6271c04947822,2011-06-07,110936,-10,4520,EVS0305732,2011-06-17,2011-06-07,2.006932,49028d1327e3e8cb_4903a584a1bcb5a6271c04947822_...,/mnt/obi0/phi/echo/npyFiles/BWH/4902/49028d132...,2011-06-07 14:46:51,49028d1327e3e8cb_4903a584a1bcb5a6271c04947822_...,BWH,Vivid7,GE Vingmed Ultrasound,0.0,40.175438,39.0,123.0,0.033248,0.033248,2.502223e-06,1.762137e-05,5.737234e-03,3.471662e-01,1.249586e-01,5.221178e-01,1.0,view_other,train
1,15369853,4a134027917d035e_4903a5858428385836eaad7bea07,2010-06-28,103286,41,282,EVS0253746,2010-05-18,2010-06-28,0.666919,4a134027917d035e_4903a5858428385836eaad7bea07_...,/mnt/obi0/phi/echo/npyFiles/BWH/4a13/4a1340279...,2010-06-28 14:27:40,4a134027917d035e_4903a5858428385836eaad7bea07_...,BWH,iE33,Philips Medical Systems,0.0,57.536000,29.0,76.0,0.052629,0.052629,2.701902e-08,6.205379e-08,1.032674e-05,8.662493e-07,4.281799e-06,9.999844e-01,1.0,view_other,train
2,4696696,4b7f0ac9b10ca017_4903a44b32ee54aa9f23728dc1c0,2008-02-25,84944,-15,3091,0731317W,2008-03-11,2007-11-19,1.084871,4b7f0ac9b10ca017_4903a44b32ee54aa9f23728dc1c0_...,/mnt/obi0/phi/echo/npyFiles/BWH/4b7f/4b7f0ac9b...,2008-02-25 10:29:13,4b7f0ac9b10ca017_4903a44b32ee54aa9f23728dc1c0_...,BWH,Vivid7,GE Vingmed Ultrasound,0.0,40.306123,50.0,99.0,0.035806,0.035806,5.339579e-11,2.314053e-09,8.073507e-09,2.499390e-10,1.028527e-08,1.000000e+00,1.0,view_other,test
3,15807142,4a134bddf3e7bda5_4903a585842c221cd9b7f2f011ca,2010-02-19,101234,2,1486,EVS0259159,2010-02-17,2010-08-04,1.557312,4a134bddf3e7bda5_4903a585842c221cd9b7f2f011ca_...,/mnt/obi0/phi/echo/npyFiles/BWH/4a13/4a134bddf...,2010-02-19 10:00:24,4a134bddf3e7bda5_4903a585842c221cd9b7f2f011ca_...,BWH,SEQUOIA,ACUSON,0.0,33.333000,71.0,57.0,1.000000,1.000000,1.272946e-02,3.574055e-03,3.889741e-04,7.146523e-04,2.943949e-05,9.825634e-01,1.0,view_other,train
4,12308375,4a14f9ea2f3e7815_4903a44ab128d8bfccf3d7874b92,2009-03-26,93883,8,2390,EVS0176928,2009-03-18,2009-03-26,1.097002,4a14f9ea2f3e7815_4903a44ab128d8bfccf3d7874b92_...,/mnt/obi0/phi/echo/npyFiles/BWH/4a14/4a14f9ea2...,2009-03-26 15:39:45,4a14f9ea2f3e7815_4903a44ab128d8bfccf3d7874b92_...,BWH,iE33,Philips Medical Systems,0.0,42.257000,71.0,70.0,0.015881,0.015881,7.960332e-14,2.769481e-12,2.892477e-12,1.173202e-11,3.757968e-11,1.000000e+00,1.0,view_other,train
5,33433715,48b60f555feba794_4903a583583f6c11bb254ec9db72,2016-08-10,148506,-12,6866,E2770907,2016-08-22,2016-08-10,1.390675,48b60f555feba794_4903a583583f6c11bb254ec9db72_...,/mnt/obi0/phi/echo/npyFiles/BWH/48b6/48b60f555...,2016-08-10 09:44:10,48b60f555feba794_4903a583583f6c11bb254ec9db72_...,BWH,iE33,Philips Medical Systems,0.0,27.107000,85.0,67.0,0.041841,0.041841,3.460966e-01,1.793437e-07,6.535502e-01,1.479856e-05,3.373438e-04,8.947133e-07,1.0,view_a4c,train
6,16040560,4a10449c483cfaf9_4903a581a72d9df6efa0d35fe3c0,2014-06-19,124887,364,5225,EVS0489042,2013-06-20,2014-06-19,1.457699,4a10449c483cfaf9_4903a581a72d9df6efa0d35fe3c0_...,/mnt/obi0/phi/echo/npyFiles/BWH/4a10/4a10449c4...,2014-06-19 13:17:18,4a10449c483cfaf9_4903a581a72d9df6efa0d35fe3c0_...,BWH,Vivid7,GE Vingmed Ultrasound,0.0,40.346509,62.0,75.0,0.056266,0.056266,4.020519e-10,5.284419e-09,9.204417e-07,2.660541e-09,6.157440e-09,9.999990e-01,1.0,view_other,test
7,8667602,4b735f430696041b_4903a58050942bdfb1c0c9835de0,2015-03-02,135831,94,3859,EVS0517876,2014-11-28,2014-09-23,1.805059,4b735f430696041b_4903a58050942bdfb1c0c9835de0_...,/mnt/obi0/phi/echo/npyFiles/BWH/4b73/4b735f430...,2015-03-02 10:59:18,4b735f430696041b_4903a58050942bdfb1c0c9835de0_...,BWH,Vivid7,GE Vingmed Ultrasound,0.0,26.367040,90.0,92.0,0.043478,0.043478,7.295377e-08,7.715913e-08,4.885771e-08,7.723793e-05,9.999222e

In [8]:
dset_filename = '210_getStressTest_files_dset_BWH_'+cfr_meta_date+'.parquet'
files_cfr_dset.to_parquet(os.path.join(meta_dir, dset_filename))